In [2]:
import requests
import json
from langchain_openai import ChatOpenAI
import os
from pydantic import BaseModel
from typing import List, Dict, Any
from langchain_core.prompts import ChatPromptTemplate
import os
import pathlib
from bs4 import BeautifulSoup
import re
from bs4 import BeautifulSoup
import re
from langchain_openai import ChatOpenAI
from pydantic import Field
from typing import Literal
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.schema import HumanMessage, AIMessage
from __future__ import print_function
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
from IPython.display import Markdown, display

In [ ]:
with open(".env", "r") as f:
  for line in f:
    key, value = line.strip().split("=")
    os.environ[key] = value


In [4]:
search_terms = ["Software Engineer New Grad", "Entry level Software Engineer", "University Graduate Software Engineer", "Junior Software Developer", "AI automation Engineer"]

In [ ]:
class ResultRelevance(BaseModel):
  explanation: str
  id: str

class RelevanceCheckOutput(BaseModel):
  relevant_results: List[ResultRelevance]

def search_serper(search_query):
  url = "https://google.serper.dev/search"
    
  payload = json.dumps({
    "q": search_query,
    "gl": "gb", 
    "num": 30,
    "tbs": "qdr:d"
  })

  headers = {
    'X-API-KEY': '0130e7da8283ec8a8d9721c6c38a97d557867923',
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  results = json.loads(response.text)
  results_list = results['organic']

  all_results = []
  for id, result in enumerate(results_list, 1):
    result_dict = {
      'title': result['title'],
      'link': result['link'],
      'snippet': result['snippet'],
      'search_term': search_query,
      'id': id
    }
    all_results.append(result_dict)
  return all_results

def load_prompt(prompt_name):
  with open(f"prompts/{prompt_name}.md", "r") as file:
    return file.read()
  
def check_search_relevance(search_results: Dict[str, Any]) -> RelevanceCheckOutput:
  prompt = load_prompt("relevance_check")
  prompt_template = ChatPromptTemplate.from_messages(
    ("system_prompt", prompt)
  )

  llm = ChatOpenAI(model="gpt-4o").with_structured_output(RelevanceCheckOutput)
  llm_chain = prompt_template | llm
  return llm_chain.invoke({'input_search_results': search_results})

relevant_results = []
for search_term in search_terms:
  python_results = search_serper(search_term)
  results = check_search_relevance(python_results)
  
  # Get the relevant result IDs from the LLM output
  relevant_ids = [r.id for r in results.relevant_results]
  
  # Filter original results to only include those with matching IDs
  filtered_results = [r for r in python_results if str(r['id']) in relevant_ids]
  
  relevant_results.extend(filtered_results)

KeyError: 'message'